In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

[NYPD_Zipcodes.csv](https://drive.google.com/file/d/1ADQ9WdHUjqzGuVnVyBOJ-5sdpjzHNxzQ/view?usp=sharing)

In [2]:
df = pd.read_csv("nyc_crime.csv")

In [36]:
df.head()

,Unnamed: 0,ROW#,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CMPLNT_FR_DT,CMPLNT_FR_TM,CMPLNT_TO_DT,CMPLNT_TO_TM,CRM_ATPT_CPTD_CD,...,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,zipcode
0,0,1,251836521,84.0,BROOKLYN,8/13/13,0:01:00,8/13/22,23:59:00,COMPLETED,...,<18,BLACK,F,988902,192641,40.695439,-73.983225,"(40.6954388081238, -73.9832253756043)",POINT (-73.9832253756043 40.6954388081238),11201.0
1,1,2,247620938,75.0,BROOKLYN,1/1/92,0:00:00,1/1/97,0:00:00,COMPLETED,...,25-44,BLACK,F,1017036,183890,40.671360,-73.881811,"(40.6713598203364, -73.8818110231735)",POINT (-73.8818110231735 40.6713598203364),11208.0
2,2,3,240876502,70.0,BROOKLYN,12/1/21,8:00:00,12/2/21,8:45:00,COMPLETED,...,65+,BLACK,F,995957,177430,40.653676,-73.957808,"(40.653676, -73.957808)",POINT (-73.957808 40.653676),11226.0
3,3,4,238808851,75.0,BROOKLYN,12/9/21,11:05:00,12/9/21,16:00:00,COMPLETED,...,25-44,ASIAN / PACIFIC ISLANDER,M,1020417,185492,40.675737,-73.869612,"(40.675737, -73.869612)",POINT (-73.869612 40.675737),11208.0
4,4,5,246251552,62.0,BROOKLYN,10/4/21,15:00:00,5/9/22,15:00:00,COMPLETED,...,65+,WHITE,F,990783,157481,40.598924,-73.976472,"(40.598924, -73.976472)",POINT (-73.976472 40.598924),11223.0


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519505 entries, 0 to 519504
Data columns (total 39 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Unnamed: 0                519505 non-null  int64  
 1   ROW#                      519505 non-null  int64  
 2   CMPLNT_NUM                519505 non-null  object 
 3   ADDR_PCT_CD               519475 non-null  float64
 4   BORO_NM                   519505 non-null  object 
 5   CMPLNT_FR_DT              519505 non-null  object 
 6   CMPLNT_FR_TM              519505 non-null  object 
 7   CMPLNT_TO_DT              481341 non-null  object 
 8   CMPLNT_TO_TM              519505 non-null  object 
 9   CRM_ATPT_CPTD_CD          519505 non-null  object 
 10  HADEVELOPT                519505 non-null  object 
 11  HOUSING_PSA               33128 non-null   float64
 12  JURISDICTION_CODE         519505 non-null  int64  
 13  JURIS_DESC                519505 non-null  o

#### Drop unused columns ####

In [38]:
df.drop(['Unnamed: 0','ROW#'], axis=1, inplace=True)

#### Handle some missing data ####

In [39]:
df.loc[df['zipcode'].isnull(), 'zipcode'] = 0
df.loc[df['PD_CD'].isnull(), 'PD_CD'] = 0
df.loc[df['ADDR_PCT_CD'].isnull(), 'ADDR_PCT_CD'] = 0

#### Convert zipcode to string ####

In [40]:
df.zipcode = df.zipcode.astype('int')
df.zipcode = df['zipcode'].apply(lambda x: str(x))

#### Concatenate Date & Time columns ####

In [41]:
df['CMPLNT_FR_DT_TM'] = df['CMPLNT_FR_DT'] + ' ' + df['CMPLNT_FR_TM']
df['CMPLNT_TO_DT_TM'] = df['CMPLNT_TO_DT'] + ' ' + df['CMPLNT_TO_TM']

df.drop(['CMPLNT_FR_DT','CMPLNT_FR_TM', 'CMPLNT_TO_DT', 'CMPLNT_TO_TM'], axis=1, inplace=True )

#### Function to reformat date/time strings ####

In [42]:
def parse_date_time (date_time):
    try:
        (newdt,newtm) = date_time.split(' ')
    except:
        newdt = date_time;
        newtm = '00:00:00'
    try:
        newdt_list = newdt.split('/')
    except:
        newdt_list = ['01', '01', '1900']
    if len(newdt_list[0]) == 1:
        newdt_list[0] = '0' + newdt_list[0]
    if len(newdt_list[1]) == 1:
        newdt_list[1] = '0' + newdt_list[1]
    if int(newdt_list[2]) < 25:
        newdt_list[2] = '20' + newdt_list[2]
    elif int(newdt_list[2]) < 100:
        newdt_list[2] = '19' + newdt_list[2]
    elif int(newdt_list[2]) < 1900:
        newdt_list[2] = '1900'
    newtm_list = newtm.split(":")
    if (len(newtm_list) < 3):
        new_date_time =  newdt_list[2] + '-' + newdt_list[0] + '/' + \
        newdt_list[1] + ' ' + '00:00:00'
        return new_date_time
#        return(dt.strptime(new_date_time, '%m/%d/%Y %H:%M:%S'))
    for i in range(len(newtm_list)):
        if len(newtm_list[i]) == 1:
            newtm_list[i] = '0' + newtm_list[i]
    new_date_time = newdt_list[2] + '-' + newdt_list[0] + '-' + \
    newdt_list[1] + ' ' + newtm_list[0] + ':' + newtm_list[1] + ':' + \
    newtm_list[2]
    return new_date_time
#    return(dt.strptime(new_date_time, '%m/%d/%Y %H:%M:%S'))

#### Reformat Date/Time columns for upload to MySQL ####

In [43]:
df['CMPLNT_FR_DT_TM'] = df['CMPLNT_FR_DT_TM'].apply(lambda x: parse_date_time(x))
df['CMPLNT_TO_DT_TM'] = df['CMPLNT_TO_DT_TM'].apply(lambda x: parse_date_time(x))
df['RPT_DT'] = df['RPT_DT'].apply(lambda x: parse_date_time(x))

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 519505 entries, 0 to 519504
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   CMPLNT_NUM                519505 non-null  object 
 1   ADDR_PCT_CD               519505 non-null  float64
 2   BORO_NM                   519505 non-null  object 
 3   CRM_ATPT_CPTD_CD          519505 non-null  object 
 4   HADEVELOPT                519505 non-null  object 
 5   HOUSING_PSA               33128 non-null   float64
 6   JURISDICTION_CODE         519505 non-null  int64  
 7   JURIS_DESC                519505 non-null  object 
 8   KY_CD                     519505 non-null  int64  
 9   LAW_CAT_CD                519505 non-null  object 
 10  LOC_OF_OCCUR_DESC         519505 non-null  object 
 11  OFNS_DESC                 519505 non-null  object 
 12  PARKS_NM                  519505 non-null  object 
 13  PATROL_BORO               519505 non-null  o

In [45]:
pd.set_option('display.max_columns', None)
dups = df[df.duplicated(subset = ['CMPLNT_NUM'], keep=False)]
dups = dups.sort_values(by=['CMPLNT_NUM',], ascending = False)
dups

,CMPLNT_NUM,ADDR_PCT_CD,BORO_NM,CRM_ATPT_CPTD_CD,HADEVELOPT,HOUSING_PSA,JURISDICTION_CODE,JURIS_DESC,KY_CD,LAW_CAT_CD,LOC_OF_OCCUR_DESC,OFNS_DESC,PARKS_NM,PATROL_BORO,PD_CD,PD_DESC,PREM_TYP_DESC,RPT_DT,STATION_NAME,SUSP_AGE_GROUP,SUSP_RACE,SUSP_SEX,TRANSIT_DISTRICT,VIC_AGE_GROUP,VIC_RACE,VIC_SEX,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lat_Lon,New Georeferenced Column,zipcode,CMPLNT_FR_DT_TM,CMPLNT_TO_DT_TM
260675,8101169H6113,32.0,MANHATTAN,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO MAN NORTH,0.0,(null),SOCIAL CLUB/POLICY LOCATI,2022-02-21 00:00:00,(null),18-24,BLACK,M,NaN,25-44,BLACK,M,1000043,234844,40.811255,-73.942946,"(40.811255, -73.942946)",POINT (-73.942946 40.811255),10037,2005-03-06 07:00:00,1900-01-01 00:00:00
259756,8101169H6113,32.0,MANHATTAN,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,INSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO MAN NORTH,0.0,(null),SOCIAL CLUB/POLICY LOCATI,2022-02-21 00:00:00,(null),18-24,BLACK,M,NaN,18-24,BLACK,M,1000043,234844,40.811255,-73.942946,"(40.811255, -73.942946)",POINT (-73.942946 40.811255),10037,2005-03-06 07:00:00,1900-01-01 00:00:00
257499,261120108H1,42.0,BRONX,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BRONX,0.0,(null),(null),2022-12-29 00:00:00,(null),18-24,BLACK HISPANIC,M,NaN,25-44,BLACK,M,1009615,241690,40.830025,-73.908341,"(40.830025, -73.908341)",POINT (-73.908341 40.830025),10456,2022-12-29 16:48:00,1900-01-01 00:00:00
258290,261120108H1,42.0,BRONX,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BRONX,0.0,(null),(null),2022-12-29 00:00:00,(null),18-24,BLACK,F,NaN,25-44,BLACK,M,1009615,241690,40.830025,-73.908341,"(40.830025, -73.908341)",POINT (-73.908341 40.830025),10456,2022-12-29 16:48:00,1900-01-01 00:00:00
257355,260637443H1,48.0,BRONX,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BRONX,0.0,(null),GROCERY/BODEGA,2022-12-28 00:00:00,(null),25-44,BLACK HISPANIC,M,NaN,45-64,BLACK HISPANIC,M,1015722,246426,40.843004,-73.886251,"(40.843004, -73.886251)",POINT (-73.886251 40.843004),10460,2022-12-28 18:36:00,1900-01-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8544,239123816H1,69.0,BROOKLYN,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BKLYN SOUTH,0.0,(null),SOCIAL CLUB/POLICY LOCATI,2022-01-13 00:00:00,(null),(null),(null),(null),NaN,25-44,BLACK,M,1009631,174082,40.644458,-73.908540,"(40.644458, -73.90854)",POINT (-73.90854 40.644458),11236,2022-01-13 03:46:00,1900-01-01 00:00:00
6596,239123816H1,69.0,BROOKLYN,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BKLYN SOUTH,0.0,(null),SOCIAL CLUB/POLICY LOCATI,2022-01-13 00:00:00,(null),(null),(null),(null),NaN,25-44,BLACK,F,1009631,174082,40.644458,-73.908540,"(40.644458, -73.90854)",POINT (-73.90854 40.644458),11236,2022-01-13 03:46:00,1900-01-01 00:00:00
149308,238885819H1,44.0,BRONX,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BRONX,0.0,(null),(null),2022-01-09 00:00:00,(null),25-44,BLACK,F,NaN,25-44,BLACK HISPANIC,M,1005825,241931,40.830694,-73.922037,"(40.830694, -73.922037)",POINT (-73.922037 40.830694),10452,2022-01-09 22:54:00,1900-01-01 00:00:00
150792,238885819H1,44.0,BRONX,COMPLETED,(null),NaN,0,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,(null),PATROL BORO BRONX,0.0,(null),(null),2022-01-09 00:00:00,(null),25-44,BLACK,M,NaN,25-44,BLACK HISPANIC,M,1005825,241931,40.830694,-73.922037,"(40.830694, -73.922037)",POINT (-73.922037 40.830694),10452,2022-01-09 22:54:00,1900-01-01 00:00:00


In [46]:
pd.reset_option('display.max_columns')

In [47]:
print(df.CMPLNT_NUM.map(lambda x: len(x)).max())

13


#### Write csv file for import to MySQL ####

In [48]:
max_rows = df.shape[0]
chunk_size=50000
start_index = 0
chunk_number = 1
end_index = chunk_size
while start_index < max_rows:
    print('outputting: ', start_index, end_index)
    df_out = df.iloc[start_index:end_index]
    file_name = 'nyc_crime' + str(chunk_number) + '.csv'
    df_out.to_csv(file_name, index=True)
    chunk_number = chunk_number + 1
    start_index = end_index
    end_index = min(start_index + chunk_size, max_rows)

outputting:  0 50000
outputting:  50000 100000
outputting:  100000 150000
outputting:  150000 200000
outputting:  200000 250000
outputting:  250000 300000
outputting:  300000 350000
outputting:  350000 400000
outputting:  400000 450000
outputting:  450000 500000
outputting:  500000 519505


In [34]:
df.to_csv('nyc_crime.csv', index=False)

In [33]:
df.CMPLNT_FR_DT_TM.unique()

array(['2013-08-13 00:01:00', '1992-01-01 00:00:00',
       '2021-12-01 08:00:00', ..., '2022-12-28 04:50:00',
       '2022-12-29 13:17:00', '2022-12-25 07:13:00'], dtype=object)

In [352]:
df.loc[['COMPLNT_NUM' == '8101169H6113']]

InvalidIndexError: [False]